In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
from Helper import *
%autoreload 2
%load_ext rpy2.ipython
#%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from helper import pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from umap import UMAP

In [ ]:
! pip install umap-learn

In [ ]:
# biggest change genes across time point
# GSEA
#counts = pd.read_csv("data/expression.MAX_AMLproject.counts.tsv", sep='\t')
counts = pd.read_csv("data/MAX_AMLproject.rsem_genes_expected_count.txt", sep='\t')
#transcripts = pd.read_csv("data/expression.MAX_AMLproject.transcripts.tsv", sep='\t')b

In [ ]:
counts.shape

In [ ]:
# we have a number of non zero similar to nb of prot coding genes
np.count_nonzero(counts.values[:,2:])/30

In [ ]:
counts.columns

In [ ]:
filtered = filterProteinCoding(listofgenes = counts['gene_id'].values, idtype = "ensembl_gene_id")
len(filtered)

In [ ]:
counts = counts.iloc[filtered]

filter some more

In [ ]:
toremove = np.argwhere(counts[counts.columns.values[2:]].values.var(1)==0)
toremove.ravel()

In [ ]:
counts = counts.drop(counts.index[toremove.ravel()],0)
counts = counts.reset_index()
counts = counts.drop(columns='index')

renormalize the data

In [ ]:
data = counts[counts.columns.values[2:]].values
data = np.log2(1+ data)

The avg amount of expressed genes in the samples

In [ ]:
np.count_nonzero(counts.values[:,2:])/30

In [ ]:
counts = pd.DataFrame(data.T,columns=counts['gene_id'],index=counts.columns.values[2:])

finding the different experiments

In [ ]:
DMSO = '1'
MS2 = '2'
JQ1 = '3'
MS2_JQ1 = '4'
START = '0'

In [ ]:
counts

In [ ]:
counts['type'] = [val[0] for val in counts.index.values]

In [ ]:
counts['DMSO'] = [val[0]==DMSO for val in counts.index.values]
counts['MS2'] = [val[0]==MS2 for val in counts.index.values]
counts['JQ1'] = [val[0]==JQ1 for val in counts.index.values]
counts['MS2_JQ1'] = [val[0]==MS2_JQ1 for val in counts.index.values]
counts['START'] = [val[0]==START for val in counts.index.values]

In [ ]:
START = counts.loc[[val for val in counts.index if START == val[0]]]
DMSO = pd.concat([START, counts.loc[[val for val in counts.index if DMSO == val[0]]]])
MS2 = pd.concat([START, counts.loc[[val for val in counts.index if MS2 == val[0]]]])
JQ1 = pd.concat([START, counts.loc[[val for val in counts.index if JQ1 == val[0]]]])
MS2_JQ1 = pd.concat([START, counts.loc[[val for val in counts.index if MS2_JQ1 == val[0]]]])

## some plotting

In [ ]:
scatter(MDS(2).fit_transform(counts.values[:,:-6]), counts.index.values, counts['type'].values)

In [ ]:
scatter(TSNE(2,10).fit_transform(counts.values[:,:-6]), counts.index.values, counts['type'].values)

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
MS2repA = MS2.loc[[val for val in MS2.index if 'A' in val[:-4]]][MS2.columns.values[:-6]]
MS2repB = MS2.loc[[val for val in MS2.index if 'B' in val[:-4]]][MS2.columns.values[:-6]]
MS2repC = MS2.loc[[val for val in MS2.index if 'C' in val[:-4]]][MS2.columns.values[:-6]]

In [ ]:
# reordering
MS2repA = MS2repA.loc[MS2repA.index.values[[0,3,4,5,1,2]]]
MS2repB = MS2repB.loc[MS2repB.index.values[[0,3,4,5,1,2]]]
MS2repC = MS2repC.loc[MS2repC.index.values[[0,3,4,5,1,2]]]

In [ ]:
MS2repA.mean(1)

In [ ]:
MS2repB.mean(1)

In [ ]:
MS2repC.mean(1)

In [ ]:
mostvargenesA = np.argsort(MS2repA.var(0).values)
mostvargenesB = np.argsort(MS2repB.var(0).values)
mostvargenesC = np.argsort(MS2repC.var(0).values)

In [ ]:
mostvargenesC

Do we have a good overlap?
- we do only if we first remove the non prot coding genes...

In [ ]:
commonvar = set(counts.columns.values[mostvargenesA[:2000]].tolist()) & set(counts.columns.values[mostvargenesB[:2000]].tolist()) & set(counts.columns.values[mostvargenesC[:2000]].tolist())
len(commonvar)

In [ ]:
DMSO = DMSO.loc[DMSO.index.values[[0,1,2,4,6,8,3,5,7]]][DMSO.columns.values[:-6]]

In [ ]:
prev = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.tpm')

In [ ]:
prev.columns

In [ ]:
DMSO = DMSO.append(pd.DataFrame(data=prev['ACH-000045'].apply(lambda x : np.log2(x+1)).tolist(),index=prev.gene_id, columns=['ACH-000045']).T,sort=True).dropna(1)

In [ ]:
sns.heatmap(DMSO.T.corr(), 
            xticklabels=DMSO.index.values,
            yticklabels=DMSO.index.values)

### A quick detour to look for clustering

In [ ]:
prev = prev.set_index("gene_id")

In [ ]:
data = pd.concat([prev.T.drop('transcript_id(s)').astype(np.float).apply(lambda x: np.log2(x+1)), DMSO])

In [ ]:
metadata = tc.get(name='sample-info-19q2-3c18', file='achiles_sample_info_full')

In [ ]:
data.shape

In [ ]:
testlabels = ['Leukemia']*10
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in data.index[:1288]]

In [ ]:
data = data.dropna(1)

In [ ]:
data.shape

In [ ]:
pca = PCA(50).fit(scale(data.values, axis=0, with_std=False))
dimred = pca.transform(scale(data.values, axis=0, with_std=False))

In [ ]:
dimred.shape

In [ ]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(dimred[:-10], trainlabels) 
predicted = neigh.predict(dimred[-10:])

In [ ]:
predicted

In [ ]:
data

In [ ]:
colors=[0]*1288
colors.extend([1,2,1,2,1,2,1,1,1,3])

In [ ]:
dimred = TSNE(2,150).fit_transform(dimred)

In [ ]:
len(colors)

In [ ]:
dimred.shape

In [ ]:
scatter(dimred, labels=trainlabels+testlabels, colors=colors, radi=0.4)

In [ ]:
MS2 = MS2.loc[MS2.index.values[[0,1,2,5,10,15,6,11,16,7,12,17,3,8,13,4,9,14]]][MS2.columns.values[:-6]]

In [ ]:
MS2red = pca.transform(scale(MS2.values, axis=0, with_std=False))

In [ ]:
sns.heatmap(np.corrcoef(MS2red), 
            xticklabels=MS2.index.values,
            yticklabels=MS2.index.values)

In [ ]:
JQ1 = JQ1[JQ1.columns.values[:-6]]

In [ ]:
JQ1red = pca.transform(scale(JQ1.values, axis=0, with_std=False))

In [ ]:
sns.heatmap(np.corrcoef(JQ1red), 
            xticklabels=JQ1.index.values,
            yticklabels=JQ1.index.values)

In [ ]:
MS2_JQ1 = MS2_JQ1[MS2_JQ1.columns.values[:-6]]

In [ ]:
MS2_JQ1red = pca.transform(scale(MS2_JQ1.values, axis=0, with_std=False))

In [ ]:
sns.heatmap(np.corrcoef(MS2_JQ1red),
            xticklabels=MS2_JQ1.index.values,
            yticklabels=MS2_JQ1.index.values)

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
genenames, notparsed = convertGenes(counts.columns.values)

In [ ]:
genenames = np.array(genenames).tolist()

In [ ]:
ctfpos = [genenames.index(val) for val in ctf if val in genenames]
notctfpos = [i for i, val in enumerate(genenames) if val not in ctf]

We find a CTF not in the dataset

In [ ]:
[val for val in ctf if val not in genenames]

In [ ]:
ctf.remove('IKAROS')

# Let's look at how they change accross time

how core transcription factors change accross time when the cell is with MS2 Condition

first line is avg count of CTF and line 2 is avg count of protein coding genes 

In [ ]:
ctf_var = MS2repA.values[:,ctfpos].mean(1)
mean_var = MS2repA.values.mean(1)

In [ ]:
sns.lineplot(data=[ctf_var, mean_var])

In [ ]:
ctf_var = MS2repB.values[:,ctfpos].mean(1)
mean_var = MS2repB.values.mean(1)

In [ ]:
sns.lineplot(data=[ctf_var, mean_var])

# we can verify per tf 

but we expect to find the same results

In [ ]:
ctf.append('all other gene coding')

In [ ]:
from bokeh.models import Legend, LegendItem

ctf_var = MS2repC.values[:,ctfpos]
mean_var = MS2repC.values[:,notctfpos].mean(1)
data = np.vstack((ctf_var.T, mean_var.T))
source = ColumnDataSource(data=dict(
    y=data.tolist(),
    x=[np.linspace(0, data.shape[1]-1, data.shape[1]).tolist()]*data.shape[0],
    color= viridis(data.shape[0]),
    name=ctf
))
p = figure()
r = p.multi_line('x','y',color='color',hover_line_color='blue', source=source)
legend = Legend(items=[LegendItem(label=val, renderers=[r], index=i) for i, val in enumerate(ctf)])
p.add_layout(legend)
p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('TF', '@name'),
]))
show(p)

## DESEQ ANALYSIS

In [ ]:
# biggest change genes across time point
# GSEA
# counts = pd.read_csv("data/expression.MAX_AMLproject.counts.tsv", sep='\t')
 counts = pd.read_csv("data/expression.MAX_AMLproject.tpm.tsv", sep='\t')
#transcripts = pd.read_csv("data/expression.MAX_AMLproject.transcripts.tsv", sep='\t')b

In [ ]:
counts = counts.iloc[filtered]

In [ ]:
#filter some more
toremove = np.argwhere(counts[counts.columns.values[2:]].values.var(1)==0)

In [ ]:
counts = counts.drop(counts.index[toremove.ravel()],0)
counts = counts.reset_index()
counts = counts.drop(columns='index')

In [ ]:
counts = counts.drop('transcript_id(s)',1)

In [ ]:
counts[counts.columns[1:]]

In [ ]:
counts[counts.columns[1:]] = counts[counts.columns[1:]].astype(float).apply(lambda x: np.log2(x+1))
counts[counts.columns[1:]] = scale(counts[counts.columns[1:]].values, axis=1, with_std=False)

In [ ]:
START = '0'
MS2 = '2'
JQ1 = '3'
MS2_JQ1 = '4'
DMSO = '1'

In [ ]:
DMSO = [val[0]==DMSO for val in counts.columns.values]
DMSO_t = [int(val.split('-')[1][:-1]) if val[0]==DMSO else 0 for val in counts.columns.values]
START = [val[0]==START for val in counts.columns.values]
MS2_t = [int(val.split('-')[1][:-1]) if val[0]==MS2 else 0 for val in counts.columns.values]
MS2_24 = [val == 24 for val in MS2_t]
MS2 = [val[0]==MS2 for val in counts.columns.values]
MS2_JQ1_t = [int(val.split('-')[1][:-1]) if val[0]==MS2_JQ1 else 0 for val in counts.columns.values]
MS2_JQ1 = [val[0]==MS2_JQ1 for val in counts.columns.values]
JQ1_t = [int(val.split('-')[1][:-1]) if val[0]==JQ1 else 0 for val in counts.columns.values]
JQ1 = [val[0]==JQ1 for val in counts.columns.values]

In [ ]:
design = pd.DataFrame(index=counts.columns.values[1:], 
                      columns=['START','DMSO','DMSO_t','MS2','MS2_t','MS2_24', 'MS2_JQ1', 'MS2_JQ1_t', 'JQ1', 'JQ1_t'], 
                      data=np.array([START, DMSO, DMSO_t, MS2, MS2_t, MS2_24, MS2_JQ1, MS2_JQ1_t, JQ1, JQ1_t]).T[1:])

In [ ]:
design.index = 'X' + design.index.astype(str).str.replace('-','.')

In [ ]:
design

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=counts,
                          design_matrix = design,
         design_formula="~ START + MS2_t",
         gene_column="gene_id")

In [ ]:
deseq.run_deseq()

In [ ]:
deseq.get_deseq_result()

In [ ]:
MS2res = deseq.deseq_result

In [ ]:
MS2res.pvalue = np.nan_to_num(np.array(MS2res.pvalue), 1)
MS2res.log2FoldChange = np.nan_to_num(np.array(MS2res.log2FoldChange), 0)

In [ ]:
MS2res.gene_id = convertGenes(MS2res.gene_id)[0]

In [ ]:
show(volcano(MS2res,tohighlight=ctf))

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=counts.drop('transcript_id(s)',1),
                          design_matrix = design,
         design_formula="~ START + MS2_24",
         gene_column="gene_id")
deseq.run_deseq()
deseq.get_deseq_result()
MS2res = deseq.deseq_result

In [ ]:
MS2res.pvalue = np.nan_to_num(np.array(MS2res.pvalue), 1)
MS2res.log2FoldChange = np.nan_to_num(np.array(MS2res.log2FoldChange), 0)
MS2res.gene_id = convertGenes(MS2res.gene_id)[0]
show(volcano(MS2res,tohighlight=ctf))

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=counts.drop('transcript_id(s)',1),
                          design_matrix = design,
         design_formula="~ START + JQ1",
         gene_column="gene_id")
deseq.run_deseq()
deseq.get_deseq_result()
JQ1res = deseq.deseq_result

In [ ]:
JQ1res.pvalue = np.nan_to_num(np.array(JQ1res.pvalue), 1)
JQ1res.log2FoldChange = np.nan_to_num(np.array(JQ1res.log2FoldChange), 0)
JQ1res.gene_id = convertGenes(JQ1res.gene_id)[0]
show(volcano(JQ1res,tohighlight=ctf))

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=counts.drop('transcript_id(s)',1),
                          design_matrix = design,
         design_formula="~ START + MS2_JQ1",
         gene_column="gene_id")
deseq.run_deseq()
deseq.get_deseq_result()
MS2_JQ1 = deseq.deseq_result

In [ ]:
MS2_JQ1.pvalue = np.nan_to_num(np.array(MS2_JQ1.pvalue), 1)
MS2_JQ1.log2FoldChange = np.nan_to_num(np.array(MS2_JQ1.log2FoldChange), 0)
MS2_JQ1.gene_id = convertGenes(MS2_JQ1.gene_id)[0]
show(volcano(MS2_JQ1,tohighlight=ctf))

## Results

It seems looking at both plottings of raw RNA data and Differential Expression analysis, that the RNA seq experiment is sound.

However it does not completly match to the theory. We can see some of the CTF as showing differential expression but they are not the most differentially expressed ones. 
This could amount to noise. 

It seems however, looking at the TNSE and MDS plots, that the samples underwent a transcriptional adaptation to the presence of different element. This state change could be attributable to the CTFs. But it seemed to be a transcriptional adaptation rather than transcriptional collapse. Moreover, given the experimental set up (using PCR, renormalizing the RNA) it might be difficult to see transcriptional collaps.

### DESEQ2

you might have seen that more complex variance analysis can be achieved. However it seems that with the little data at hand and the assumption that these more complex models will achieve, staying on a simpler differential expression formula allows better results. I have tried a few and it seemed to confirm this suspicion (or I would need to provide a better model than DESEQ has to offer) limma.voom?